In [232]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

img = cv.imread('images/house.jpg', cv.IMREAD_GRAYSCALE)
# img = cv.resize(img, (700, 500))
plt.hist(img.ravel(), bins = 255)
plt.show()

res, img1 = cv.threshold(img, 100, 255, cv.THRESH_BINARY_INV)
res, img2 = cv.threshold(img, 100, 255, cv.THRESH_TRUNC)
res, img3 = cv.threshold(img, 100, 255, cv.THRESH_TOZERO)

plt.figure()
images = [img, img1, img2, img3]
title = ['house', 'THRESH_BINARY_INV', 'THRESH_TRUNC', 'THRESH_TOZERO']
for i in range(4) :
    plt.subplot(2, 2, i + 1)
    plt.imshow(images[i], 'gray')
    plt.title(title[i])
    plt.tight_layout()
    plt.xticks([]), plt.yticks([])
plt.show()

Using matplotlib backend: Qt5Agg


In [233]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
img = cv.imread('images/wall.png', cv.IMREAD_GRAYSCALE)
kernel = np.ones((10, 10)).astype(np.int32)
Tophat = cv.morphologyEx(img, cv.MORPH_TOPHAT, kernel)
Blackhat = cv.morphologyEx(img, cv.MORPH_BLACKHAT, kernel)
title = ['Wall', 'TopHat', 'BlackHat']
images = [img, Tophat, Blackhat]
for i in range(3) :
    plt.subplot(1, 3, i + 1)
    plt.imshow(images[i], 'gray')
    plt.title(title[i])
    plt.tight_layout()
    plt.xticks([]), plt.yticks([])
plt.show()

Using matplotlib backend: Qt5Agg


In [234]:
import cv2 as cv 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib
img = cv.imread('images/morph_test.png', cv.IMREAD_GRAYSCALE)
kernel = np.ones((7, 7)).astype(np.int32)
img1 = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
img2 = cv.bitwise_xor(img, img1)
title = ['Original', 'Denoising', 'Defect']
images = [img, img1, img2]
for i in range(3) :
    plt.subplot(1, 3, i + 1)
    plt.imshow(images[i], 'gray')
    plt.title(title[i])
    plt.tight_layout()
    plt.xticks([]), plt.yticks([])
plt.show()

Using matplotlib backend: Qt5Agg


In [235]:
import copy 
import cv2 as cv 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib

house = cv.imread('images/house.jpg', cv.IMREAD_GRAYSCALE)
house = cv.resize(house, (700, 500))
def P3_1(img) :
    cimg = copy.deepcopy(img)
    Res = np.ones(255)
    std = np.zeros(255)
    for thd in range(0, 255) :
        arr1 = cimg[np.where(img > thd)]
        arr2 = cimg[np.where(img <= thd)]
        w1 = len(arr1)
        w2 = len(arr2)
        m1 = np.sum(arr1) / w1
        m2 = np.sum(arr2) / w2
        m = (m1 * w1 + m2 * w2) / (w1 + w2)
        #组内方差和
        std_ingroud = np.sum(np.square(arr1 - m1)) + np.sum(np.square(arr2 - m2))
        #组间方差和
        std_outgroud = (m1 - m) * (m1 - m) * w1 + (m2 - m) * ( m2 - m) * w2
        #同时考虑组内和组间方差，当组内方差越小、组间方差越大时效果越好。
        res = std_outgroud / std_ingroud
        std[thd] = std_outgroud
        Res[thd] = res
    plt.plot(list(range(255)), Res, 'r-')
    plt.show()
    thd = np.argmax(Res[ : ])
    cimg[np.where(cimg >= thd)] = 255
    cimg[np.where(cimg < thd)] = 0
    return cimg, thd
        
img2, thd = P3_1(house)
ret, img3 = cv.threshold(house, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

images = [house, img2, img3]
title = ['original', f'my threshold {thd}', f'cv threshold {int(ret)}']
for i in range(3) :
    plt.subplot(1, 3, i + 1)
    plt.imshow(images[i], 'gray')
    plt.title(title[i])
    plt.xticks([]), plt.yticks([])
    plt.tight_layout()
plt.show()
print(f'thd : {thd}, ret : {ret}')

Using matplotlib backend: Qt5Agg
thd : 97, ret : 97.0


In [237]:
import cv2 as cv 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib

img = cv.imread('images/PCB.png', cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
r, g, b = cv.split(img)

kernel1 = np.zeros((16, 16)).astype(np.int32)
kernel2 = np.zeros((6, 6)).astype(np.int32)
#圆心
center_radius1 = int(16 / 2)
center_radius2 = int(6 / 2)
#做圆形卷积核
cv.circle(kernel1, (center_radius1, center_radius1), center_radius1, (255, 255, 255), -1, cv.LINE_AA)
cv.circle(kernel2, (center_radius2, center_radius2), center_radius2, (255, 255, 255), -1, cv.LINE_AA)

kernel1 = kernel1.astype(np.uint8)
kernel2 = kernel2.astype(np.uint8)

_, img_binary = cv.threshold(b, 180, 255, cv.THRESH_BINARY)
img_open = cv.morphologyEx(img_binary, cv.MORPH_OPEN, kernel2)
img_dilate = cv.dilate(img_close, kernel1, 10)

#焊点个数
Solder_joint = 0
#直线检测，以行为迭代对象，如果如果遇到白点则记录到直线中
#因为焊点之间是分开的，则直线中不为零的某一段就代表一个焊点
Detection_line = np.zeros(img_dilate.shape[1])
for row in range(img_dilate.shape[0]) :
    if row == img_dilate.shape[0] / 2 or row == img_dilate.shape[0] - 1:#总统计焊点两次
        flag = 0
        for i in range(len(Detection_line)) :
            if Detection_line[i] == 0 and flag == 1 :#如果当前点是黑点且是由白到黑点的跨越，则代表着一个焊点
                Solder_joint += 1
                flag = 0
            if Detection_line[i] != 0 :#由白点进入到黑点，记录flag为1
                flag = 1
        Detection_line[ : ] = 0#统计完一次黑点对直线清零
    Detection_line = Detection_line + img_dilate[row]
    
print(f'焊点个数:{Solder_joint}')
cv.imshow('img_dilate', img_dilate)
cv.waitKey(0)

Using matplotlib backend: Qt5Agg
焊点个数:40


-1